In [ ]:
# import pandas as pd 
import pandas as pd 
import numpy as np
  
df1 = pd.read_csv('SVI_2020_US_county.csv')
df2 = pd.read_csv('NRI_Table_Counties/NRI_Table_Counties.csv') 
  
# merging df1 and df2 by ID 
# i.e. the rows with common ID's get merged 
# with all the ID's of right dataframe i.e. df2 
# and NaN values for df1 columns where ID do not match 
df = pd.merge(df1, df2, on="FIPS", how="right") 
df_cleaned = df[['ST','STATE_x','STCNTY', 'COUNTY_x','FIPS',\
                 'EP_POV150','EP_HBURD','EP_DISABL','EP_SNGPNT','EP_AGE17','EP_AGE65',\
                 'EP_MINRTY','EP_AFAM','EP_HISP','EP_ASIAN','EP_AIAN','EP_NHPI',\
                 'RISK_VALUE','EAL_VALT',
                 'RISK_SCORE','EAL_SCORE','CFLD_RISKS','DRGT_RISKS','HWAV_RISKS',\
                 'HRCN_RISKS','RFLD_RISKS','WFIR_RISKS',\
                    'CFLD_RISKV','HWAV_RISKV',]]

df_cleaned = df_cleaned[df_cleaned['STATE_x'].notna()]
df_cleaned.fillna(0, inplace=True)
df_cleaned.head()

# df_cleaned_2 = df_cleaned[['EP_POV150','EP_HBURD','EP_AFAM','EP_HISP','RISK_VALUE']]
df_cleaned.to_csv('nri-svi-cleaned.csv', sep=',', index=False, encoding='utf-8')
# df_cleaned_2.to_csv('nri-svi-cleaned-2.csv', sep=',', index=False, encoding='utf-8')

In [ ]:
# creating dataframes as df1 and df2 
strong2 = pd.read_csv('STRONG-II/STRONGII_nonTX.csv')
strong1 = pd.read_csv('STRONG-I/STRONG_Version_2_Updated_2021-12-01/STRONG_Version_2_Updated_2021-12-01.csv') 


# merging df1 and df2 by ID 
# i.e. the rows with common ID's get merged 
# with all the ID's of right dataframe i.e. df2 
# and NaN values for df1 columns where ID do not match 
strong = pd.merge(strong1, strong2, on="respid", how="right") 
# print(strong.describe())
strong_cleaned = strong[['h2', 'storm3_1','storm3_2','storm3_3','storm3_4','storm3_5',\
                         'storm3a_1','storm3a_1','storm3a_2','storm3a_3','storm3a_4','storm3a_5',\
                 'danger1','danger2','danger3','danger4','danger5',\
                 'shelter1_1','shelter1_2','shelter1_3','shelter1_4','shelter1_5','shelter1_6',\
                 'dam1','dam6','help1','rec1','rec2','rec4',\
                 'hurr1','hurr2','hurr3','hurr4','hurr5','hurr6','hurr7','hurr8',\
                 'job1_y','job1a','job1b','job1c','job2',\
                 'sw1','sw2','sw3','sw4','sw5',\
                 'ph1_y','ph2_y','audit1_y','audit2_y','audit3_y',\
                 'eff1','eff2','eff3','eff4','eff5','eff6',\
                 'phq1_y','phq2_y','phq3','phq4','phq5','phq6','phq7','phq8',\
                 'pcl1','pcl2','pcl3','pcl4','pcl5','pcl6','pcl7','pcl8','pcl9','pcl10',\
                 'pcl11','pcl12','pcl13','pcl14','pcl15','pcl16','pcl17','pcl18','pcl19','pcl20',\
                 'sex','age_y','income','same_respondent','hispanic','white','black','aian','asian','pacific']]


strong_trans = pd.DataFrame(columns = ['injured','home_damage','rec_household','rec_community','adversity_score',\
                                       'lost_job','eff_score','depression_score','ptsd_score',\
                                    'sex','age','income','white','black','aian','asian','hispanic','pacific'])


strong_cleaned = strong_cleaned[strong_cleaned['storm3_1'].notna()]
strong_cleaned = strong_cleaned.loc[(strong_cleaned[['same_respondent']] != 0).all(axis=1)]

strong_cleaned['storm_living'] = strong_cleaned[['storm3_1','storm3_2','storm3_3','storm3_4','storm3_5']].sum(axis='columns')
strong_cleaned['storm_present'] = strong_cleaned[['storm3a_1','storm3a_2','storm3a_3','storm3a_4','storm3a_5']].sum(axis='columns')
strong_cleaned['storm_living_bin'] =  (strong_cleaned['storm_living']>0).astype(int)
strong_cleaned['storm_present_bin'] =  (strong_cleaned['storm_present']>0).astype(int)
# strong_cleaned['storm_affected'] = strong_cleaned[['storm_living','storm_present']].sum(axis='columns')
strong_cleaned = strong_cleaned.loc[(strong_cleaned[['storm_living','storm_present']] != 0).all(axis=1)]


# calculate adversity score by summing over relevant questions and bin into low medium high
strong_cleaned['adversity_score'] = strong_cleaned[['hurr1','hurr2','hurr3','hurr4','hurr5','hurr6','hurr7','hurr8']].sum(axis='columns')
strong_cleaned['adversity_score'][strong_cleaned['adversity_score'] > 200] = np.nan
adversity_bins = [0,1,3,strong_cleaned['adversity_score'].max()][1:]
labels=['medium adversity', 'high adversity']
strong_trans['adversity_score'] = pd.cut(strong_cleaned['adversity_score'], adversity_bins,labels=labels,include_lowest=True)
# print(strong_cleaned['adversity_score'].min())
# print(strong_cleaned['adversity_score'].max())
# print(adversity_bins)

# calculate efficacy score by summing over relevant questions and bin into low medium high
strong_cleaned['eff_score'] = strong_cleaned[['eff1','eff2','eff3','eff4','eff5','eff6']].sum(axis='columns')
strong_cleaned['eff_score'][strong_cleaned['eff_score'] > 200] = np.nan
eff_bins = np.linspace(strong_cleaned['eff_score'].min(),strong_cleaned['eff_score'].max(),4)[0:-1]
labels=['low efficacy', 'medium efficacy']
strong_trans['eff_score'] = pd.cut(strong_cleaned['eff_score'], eff_bins,labels=labels,include_lowest=True)
print(strong_cleaned['eff_score'].min())
print(strong_cleaned['eff_score'].max())
print(eff_bins)

# calculate depression score by summing over relevant questions and bin into low medium high
strong_cleaned['depression_score'] = strong_cleaned[['phq1_y','phq2_y','phq3','phq4','phq5','phq6','phq7','phq8']].sum(axis='columns')
strong_cleaned['depression_score'][strong_cleaned['depression_score'] > 200] = np.nan
depression_bins = np.linspace(strong_cleaned['depression_score'].min(),strong_cleaned['depression_score'].max(),4)[1:]
labels=['medium depression', 'high depression']
strong_trans['depression_score'] = pd.cut(strong_cleaned['depression_score'], depression_bins,labels=labels,include_lowest=True)
# print(strong_cleaned['depression_score'].min())
# print(strong_cleaned['depression_score'].max())
# print(depression_bins)

# calculate ptsd score by summing over relevant questions and bin into low medium high
strong_cleaned['ptsd_score'] = strong_cleaned[['pcl1','pcl2','pcl3','pcl4','pcl5','pcl6','pcl7','pcl8','pcl9','pcl10',\
                 'pcl11','pcl12','pcl13','pcl14','pcl15','pcl16','pcl17','pcl18','pcl19','pcl20']].sum(axis='columns')
strong_cleaned['ptsd_score'][strong_cleaned['ptsd_score'] > 200] = np.nan
ptsd_bins = np.linspace(strong_cleaned['ptsd_score'].min(),strong_cleaned['ptsd_score'].max(),4)[1:]
labels=['medium ptsd', 'high ptsd']
strong_trans['ptsd_score'] = pd.cut(strong_cleaned['ptsd_score'], ptsd_bins,labels=labels,include_lowest=True)
# print(strong_cleaned['ptsd_score'].min())
# print(strong_cleaned['ptsd_score'].max())
# print(ptsd_bins)

# mapping ints to descriptions
# d_housing = {1:'one-family house',2:'two-family house',3:'apartment',4:'mobile home',5:'row house'}
# strong_trans['housing'] = strong_cleaned['h2'].map(d_housing)

# d_storm_living = {0:'not_storm_living',1:'storm_living'}
# d_storm_present = {0:'not_storm_present',1:'storm_present'}
# strong_trans['storm_living'] = strong_cleaned['storm_living_bin'].map(d_storm_living)
# strong_trans['storm_present'] = strong_cleaned['storm_present_bin'].map(d_storm_present)


d_injured = {1:'minor injury',2:'major injury'}
strong_trans['injured'] = strong_cleaned['danger1'].map(d_injured)

d_damage = {1: 'damaged, livable', 2: 'damaged, not livable', 3: 'destroyed'}
strong_trans['home_damage'] = strong_cleaned['dam1'].map(d_damage)

d_lost_job = {2: "lost job"}
strong_trans['lost_job'] = strong_cleaned['job1_y'].map(d_lost_job)

d_rec_household = {0: "no household recovery"}
strong_trans['rec_household'] = strong_cleaned['rec1'].map(d_rec_household)

d_rec_community = {0: "no community recovery"}
strong_trans['rec_community'] = strong_cleaned['rec2'].map(d_rec_community)

d_sex = {1:'male',2:'female',3:'trans male',4:'trans female',5:'gnc'}
strong_trans['sex'] = strong_cleaned['sex'].map(d_sex)

d_age = {1:'18-25',2:'26-50',3:'51-75',4:'over 75'}
strong_trans['age'] = strong_cleaned['age_y'].map(d_age)

d_income = {1:'under 10k',2:'10k-20k',3:'20k-30k',4:'30k-40k',5:'40k-50k'}
strong_trans['income'] = strong_cleaned['income'].map(d_income)

races_ethnicities = ['white','black','aian','asian','hispanic','pacific']

for race in races_ethnicities:
        d_race = {1:race}
        strong_trans[race] = strong_cleaned[race].map(d_race)


strong_trans.to_csv('strong_transaction.csv', sep=',', index=False, encoding='utf-8')